In [1]:
print("OK")

OK


In [2]:
%pwd

'c:\\Users\\91832\\Desktop\\Generative_AI_Medical_Chatbot\\Medical_ChatBot\\research'

In [3]:
import os 
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\91832\\Desktop\\Generative_AI_Medical_Chatbot\\Medical_ChatBot'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [7]:
extracted_data=load_pdf_file(data='Data/')

In [47]:
#extracted_data

In [9]:
#Split the Data into Text Chucks

def test_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [10]:
text_chunks=test_split(extracted_data)
print("length of text chucks",len(text_chunks))

length of text chucks 5860


In [11]:
from langchain.embeddings import HuggingFaceEmbeddings


In [12]:
!pip show sentence-transformers


Name: sentence-transformers
Version: 2.2.2
Summary: Multilingual text embeddings
Home-page: https://github.com/UKPLab/sentence-transformers
Author: Nils Reimers
Author-email: info@nils-reimers.de
License: Apache License 2.0
Location: c:\users\91832\desktop\generative_ai_medical_chatbot\medical_chatbot\medicalbot\lib\site-packages
Requires: huggingface-hub, nltk, numpy, scikit-learn, scipy, sentencepiece, torch, torchvision, tqdm, transformers
Required-by: 


In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_hugging_face_embedding():
    embedding = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embedding


In [14]:
!where python

c:\Users\91832\Desktop\Generative_AI_Medical_Chatbot\Medical_ChatBot\medicalbot\Scripts\python.exe
C:\Users\91832\AppData\Local\Programs\Python\Python310\python.exe


In [15]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_hugging_face_embedding():
    """Download and return Hugging Face embeddings model."""
    embedding = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embedding

# Call the function in a separate cell when needed
embeddings= download_hugging_face_embedding()
print(" Embeddings Model Loaded Successfully!")

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_1800\1181417906.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\91832\Desktop\Generative_AI_Medical_Chatbot\Medical_ChatBot\medicalbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 Embeddings Model Loaded Successfully!


In [16]:
query_result = embeddings.embed_query("hello World")
print("length", len(query_result))

length 384


In [48]:

#query_result

In [32]:
from dotenv import load_dotenv
import os
load_dotenv




<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [33]:
PINECONE_API_KEY= os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [20]:
from pinecone import Pinecone, ServerlessSpec

from pinecone import  ServerlessSpec 
import os

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "quickstart"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [45]:
import os
os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY
os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY

In [46]:
from dotenv import load_dotenv
import os

# Ensure loading from the correct directory
env_path = os.path.abspath(".env")  # Ensure absolute path
print(f"Loading .env from: {env_path}")

# Load .env
load_dotenv(dotenv_path=env_path)

# Get API keys
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')



Loading .env from: c:\Users\91832\Desktop\Generative_AI_Medical_Chatbot\Medical_ChatBot\.env


In [25]:
from langchain_pinecone import PineconeVectorStore
#Embed each chuck and upsert the embeddings into your pinecone index.
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [26]:
#load existing index 
from langchain_pinecone import PineconeVectorStore
docsearch= PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [27]:
docsearch

In [28]:
retriever = docsearch.as_retriever(search_type= 'similarity',search_kwargs={"k":3})

In [29]:
retriever_doc = retriever.invoke("what is acne")

In [30]:

retriever_doc 

[Document(id='f725fad4-760c-42f3-b387-4912cd6249eb', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='d985d581-9f99-4326-91f1-4e1ab6d0ad82', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 37.0, 'page_label': '38', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged with oil, dead skin\ncells, and bacteri

In [36]:
from langchain_openai import OpenAI
llm=OpenAI(temperature=0.4,max_tokens=500)

In [37]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (

    "You are an assistant for Question-Answering tasks."
    "Use the following pieces of retrieved context to answer "
    "the question. if you don't know the answer ,say that you " 
    "don't know. use three sentences maximum and keep the "
    "answer concise ."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human", "{input}"),
    ]

)


In [38]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

In [44]:
response = rag_chain.invoke({"input": "what is  Acetaminophen Recommended dosage?"})
print (response["answer"])



The recommended dosage for acetaminophen varies depending on age, but for adults and children over 12 years old, it is typically 325-650 mg every 4-6 hours as needed, with a maximum of 4 grams (4000 mg) in 24 hours. Children ages 6-11 years should take 150-300 mg, three to four times a day, and children under 6 years old should consult a physician for appropriate dosages. It is important to never exceed the recommended dosage unless directed by a healthcare professional.
